In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss

In [188]:
df = pd.read_csv("..//Labs//lab2_logistic_regression//SAheart.data", sep = ",")
df = df.drop("row.names", axis = 1)
df["famhist_present"] = pd.get_dummies(df.famhist)["Present"] # famhist can be: present | absent
df.drop("famhist", axis = 1, inplace = True)
df["intercept"] = 1

X = df.drop("chd", axis = 1, inplace=False)
y = df.chd

In [172]:
df = pd.read_csv("data.simple.train.10000.csv")
X = df.iloc[:,:2]
y = pd.get_dummies(df.cls, drop_first = True)

In [20]:
df.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist_present,intercept
0,160,12.00,5.73,23.11,49,25.30,97.20,52,1,1,1
1,144,0.01,4.41,28.61,55,28.87,2.06,63,1,0,1
2,118,0.08,3.48,32.28,52,29.14,3.81,46,0,1,1
3,170,7.50,6.41,38.03,51,31.99,24.26,58,1,1,1
4,134,13.60,3.50,27.78,60,25.99,57.34,49,1,1,1


In [189]:
class logit():
    def __init__(self, exog, endog):
        self.y = endog # T x 1
        self.X = exog # T x k
        self.b = np.linalg.inv(self.X.T @ self.X) @ self.X.T @ self.y # k x 1, initialized with LS estimator
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    def cross_entrophy(y_true, y_pred):
        return y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)
    
    
    def cross_entrophy_der(X, y, b):
        p_hat = 1 / (1 + np.exp( - X @ b)) # T x 1
        return (p_hat - y) * X  # T x k
    
    def predict_proba(self, X):
        return 1 / (1 + np.exp( - X @ self.b))
        
    def predict_y(self, X, threshold = 0.5):
        p_hat = 1 / (1 + np.exp( - X @ self.b))
        return np.array([1 if i[0] > threshold else 0 for i in p_hat]).reshape(-1,1)
    
    
    def accuracy(self, X, y):
        y_hat = logit.predict_y(self, X)
        correct_num = np.sum(y_hat == y.reshape(-1,1))
        return correct_num / X.shape[0]
    
    
    def fit(self, stop_condition = 0.0001, lr = 0.0001, GD = True, SGD = False, batch_size = None, printe = False):
        loss = []
        b_updates = []
        predicted = logit.predict_proba(self, self.X)
        loss.append(logit.cross_entrophy(self.y, predicted))
        cond = 1
        i = 1
        
        if GD:
            batch_size = self.X.shape[0]
        elif SGD:
            batch_size = 1
        
        while cond > stop_condition and i < 300:
            
            
            nb = 0 #starter for batch
            

            while(nb < self.X.shape[0] and cond > stop_condition):
                
                #partitionig of dataset
                x_part = self.X[nb:(nb + batch_size), :]
                y_part = self.y[nb:(nb + batch_size), :]
            
                
                nb = nb + batch_size
                
                
                predicted = logit.predict_proba(self, x_part)
                if SGD:
                    gradient = logit.cross_entrophy_der(x_part, y_part, self.b).reshape(-1,1)
                else:
                    gradient = np.mean(logit.cross_entrophy_der(x_part, y_part, self.b), axis = 0).reshape(-1,1)
                
                # updating weights
                beta_old = self.b
                self.b = beta_old - lr * gradient
                
                # update the value of condition
                cond = np.max(np.abs((beta_old - self.b) / beta_old))
                
        

            # loss function after one full iteration
            predicted = logit.predict_proba(self, self.X)
            loss.append(logit.cross_entrophy(self.y, predicted))
            
            
            if printe == True:
                print("iteration", i, "loss function value =", loss[-1])
            i+=1

        print(cond)
        return loss
    

In [192]:
model = logit(X.to_numpy(), y.to_numpy().reshape(-1,1))
model.fit(batch_size = 20, printe = False)
model.accuracy(model.X, model.y)


0.0027982518441916236


0.696969696969697

In [191]:
from statsmodels.discrete.discrete_model import Logit

#X = df.drop("chd", axis = 1, inplace=False)
#y = df.chd

model = Logit(endog = y, exog = X)
res = model.fit()
np.sum((res.predict(X ) > 0.5) == y) / X.shape[0]

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


0.7337662337662337